Notebook para criação de chaves primárias e secundárias

# Iniciando ambiente

## Import das bibliotecas

In [15]:
import findspark
from pyspark.sql import SparkSession
import sys
import os
from pandas import read_excel

## Incialização do spark

In [2]:
findspark.init()

## Criação da instância do Spark

In [3]:
spark =  SparkSession.builder\
            .master('local')\
            .appName('dados_enem')\
            .config("spark.executor.extraClassPath", os.path.abspath('./'))\
            .getOrCreate()

22/11/07 16:42:53 WARN Utils: Your hostname, Mac-Mikeias.local resolves to a loopback address: 127.0.0.1; using 192.168.0.138 instead (on interface en2)
22/11/07 16:42:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/07 16:42:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Dados de conexão com banco de dados

In [58]:
url = f"jdbc:postgresql://localhost:5432/db_enem"
user = os.getenv('USER_DB')
password = os.getenv('PASSWORD_DB')

# Inclusão de municipio

In [7]:
df  =  spark.read.csv("/Users/mikeiasoliveira/Downloads/02 - Outros/microdados_enem_2020/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", 
                    header=True, 
                    inferSchema=True,
                    sep=';')


In [12]:
df_mun = df.select('CO_MUNICIPIO_ESC').distinct().dropna()

In [35]:
mun_IBGE = read_excel(os.path.abspath('../DTB_2021/RELATORIO_DTB_BRASIL_MUNICIPIO.xls'))

In [36]:
mun_IBGE = spark.createDataFrame(mun_IBGE)

/Users/mikeiasoliveira/Documents/Projetos/análise_dados_enem/analise-dados-enem/analise-enem-env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/Users/mikeiasoliveira/Documents/Projetos/análise_dados_enem/analise-dados-enem/analise-enem-env/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [37]:
mun_IBGE.printSchema()

root
 |-- UF: long (nullable = true)
 |-- Nome_UF: string (nullable = true)
 |-- Região Geográfica Intermediária: long (nullable = true)
 |-- Nome Região Geográfica Intermediária: string (nullable = true)
 |-- Região Geográfica Imediata: long (nullable = true)
 |-- Nome Região Geográfica Imediata: string (nullable = true)
 |-- Mesorregião Geográfica: long (nullable = true)
 |-- Nome_Mesorregião: string (nullable = true)
 |-- Microrregião Geográfica: long (nullable = true)
 |-- Nome_Microrregião: string (nullable = true)
 |-- Município: long (nullable = true)
 |-- Código Município Completo: long (nullable = true)
 |-- Nome_Município: string (nullable = true)



In [38]:
mun_IBGE = mun_IBGE.withColumnRenamed('Região Geográfica Intermediária', 'regiao_geo_inter')\
                    .withColumnRenamed('Nome Região Geográfica Intermediária', 'nome_regiao_geo_inter')\
                    .withColumnRenamed('Região Geográfica Imediata', 'regiao_geo_imed')\
                    .withColumnRenamed('Nome Região Geográfica Imediata', 'nome_regiao_geo_imed')\
                    .withColumnRenamed('Mesorregião Geográfica', 'meso_geo')\
                    .withColumnRenamed('Nome_Mesorregião', 'nome_meso')\
                    .withColumnRenamed('Microrregião Geográfica', 'micro_geo')\
                    .withColumnRenamed('Nome_Microrregião', 'nome_micro')\
                    .withColumnRenamed('Município', 'municipio')\
                    .withColumnRenamed('Código Município Completo', 'cod_mun')\
                    .withColumnRenamed('Nome_Município', 'nome_mun')\
                    .withColumnRenamed('UF', 'uf')\
                    .withColumnRenamed('Nome_UF', 'nome_uf')

In [59]:
mun_IBGE.write.format("jdbc").mode("append").option("url", url)\
                .option("dbtable", "enem2020.regiao")\
                .option("user", user)\
                .option("password", password)\
                .option("driver", "org.postgresql.Driver")\
                .save()

In [72]:
uf_IBGE = mun_IBGE.select('uf', 'nome_uf')
uf_IBGE = uf_IBGE.withColumnRenamed('uf', 'cod_uf')

In [73]:
uf_IBGE = uf_IBGE.drop_duplicates(subset=['cod_uf'])

In [77]:
uf_IBGE.select('*').show(200)

+------+-------------------+
|cod_uf|            nome_uf|
+------+-------------------+
|    11|           Rondônia|
|    12|               Acre|
|    13|           Amazonas|
|    14|            Roraima|
|    15|               Pará|
|    16|              Amapá|
|    17|          Tocantins|
|    21|           Maranhão|
|    22|              Piauí|
|    23|              Ceará|
|    24|Rio Grande do Norte|
|    25|            Paraíba|
|    26|         Pernambuco|
|    27|            Alagoas|
|    28|            Sergipe|
|    29|              Bahia|
|    31|       Minas Gerais|
|    32|     Espírito Santo|
|    33|     Rio de Janeiro|
|    35|          São Paulo|
|    41|             Paraná|
|    42|     Santa Catarina|
|    43|  Rio Grande do Sul|
|    50| Mato Grosso do Sul|
|    51|        Mato Grosso|
|    52|              Goiás|
|    53|   Distrito Federal|
+------+-------------------+



In [ ]:
uf_IBGE.write.format("jdbc").mode("append").option("url", url)\
                .option("dbtable", "enem2020.uf")\
                .option("user", user)\
                .option("password", password)\
                .option("driver", "org.postgresql.Driver")\
                .save()